# Problema

**Classificador de Chamados**

A QuantumFinance tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para algum uma área especialista no assunto para uma melhor tratativa.​

1. Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do seu modelo.​ O modelo precisar atingir um score na métrica F1 Score superior a 75%. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

2. Utilizar ao menos uma aplicação de modelos de GenAI (LLM´s) para criar o modelo classificador com os mesmos critérios do item 1.

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

Composição da nota:​

50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, aplicações de GenIA, organização do pipeline, etc.)​

50% - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com a Métrica F1 Score). Separar o pipeline completo do modelo campeão conforme template.​

O trabalho poderá ser feito em grupo de 2 até 4 pessoas (mesmo grupo do Startup One) e trabalhos iguais serão descontado nota e passível de reprovação.

Grupo:
- Luiz Henrique (352631)
- Gildo Moraes (352486)
- Igor Mazzeto (352368)

Professor:
- Anderson Vieira Dourado

# 1. Bibliotecas, Ambiente e Funções Auxiliares

## 1.1 Instalação

In [1]:
!python -m spacy download pt_core_news_md --quiet
!python -m spacy download pt_core_news_lg --quiet
!python -m spacy download pt_core_news_sm --quiet
!pip install nltk --quiet
!pip install gensim==4.3.2 --quiet
!pip install transformers torch scikit-learn --quiet
!pip install sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 11.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 16.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load

In [2]:
# Word2Vec SkipGram
!wget 'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/skip_s300.zip' #Download do arquivo no repositório
!unzip 'skip_s300.zip' # Descompactação do arquivo
!ls -la

# Word2Vec Cbow
!wget 'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/cbow_s300.zip' #Download do arquivo no repositório
!unzip 'cbow_s300.zip' # Descompactação do arquivo
!ls -la

--2024-06-23 14:41:18--  https://dados-ml-pln.s3-sa-east-1.amazonaws.com/skip_s300.zip
Resolving dados-ml-pln.s3-sa-east-1.amazonaws.com (dados-ml-pln.s3-sa-east-1.amazonaws.com)... 16.12.1.2, 3.5.232.185, 3.5.234.180, ...
Connecting to dados-ml-pln.s3-sa-east-1.amazonaws.com (dados-ml-pln.s3-sa-east-1.amazonaws.com)|16.12.1.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 958619745 (914M) [application/zip]
Saving to: ‘skip_s300.zip’

skip_s300.zip       100%[===================>] 914.21M  3.07MB/s    in 3m 23s  

2024-06-23 14:44:43 (4.49 MB/s) - ‘skip_s300.zip’ saved [958619745/958619745]

Archive:  skip_s300.zip
  inflating: skip_s300.txt           
total 3529588
drwxr-xr-x 1 root root       4096 Jun 23 14:44 .
drwxr-xr-x 1 root root       4096 Jun 23 13:15 ..
drwxr-xr-x 4 root root       4096 Jun 20 18:45 .config
drwxr-xr-x 2 root root       4096 Jun 23 14:39 .ipynb_checkpoints
drwxr-xr-x 1 root root       4096 Jun 20 18:46 sample_data
-rw-r--r-- 1 root r

## 1.2 Importação das Bibliotecas

In [3]:
# Pacotes Básicos
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split

# Pacotes NLP
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from nltk.stem.rslp import RSLPStemmer
rslp = RSLPStemmer()
import spacy
import pt_core_news_lg
import pt_core_news_sm
nlp_stops = spacy.load('pt_core_news_sm')
nlp = spacy.load('pt_core_news_lg')

from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
import pickle

from sentence_transformers import SentenceTransformer
from sklearn import preprocessing
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import torch

# Algoritmos de Vetorização NLP
from sklearn.feature_extraction.text import TfidfVectorizer #TF/TF-IDF
from sklearn.feature_extraction.text import CountVectorizer #BoW
from gensim.models import KeyedVectors #Word2Vec
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Algoritmos de Classificação
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

# Avaliação do Modelo
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## 1.3 Funções/Configurações do Notebook

In [4]:
# Função Lematização Completa do Documento
def fn_lematiza_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        sent.append(word.lemma_)
    return ' '.join(sent)

# Função Lematização dos Verbos do Documento
def fn_lematiza_verb_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        if word.pos_ =='VERB':
            sent.append(word.lemma_)
        else:
            sent.append(word.text)
    return " ".join(sent)

# Função Stemização
def fn_stem(texto):
  return ' '.join([rslp.stem(token) for token in texto])

# Função para Limpar documento
def fn_limpa_texto(texto):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    texto = texto.translate(table)
    texto = texto.strip().lower()
    texto = "".join([w for w in texto if not w.isdigit()])
    texto = word_tokenize(texto)
    texto = [y for y in texto if len(y) > 2]
    return " ".join([t for t in texto])
    return texto

In [5]:
# Função para Obter Embeddings médios para cada Texto
def average_vector(words):
  vectors = [vectorizer[word] for word in words if word in vectorizer]
  if vectors:
    return sum(vectors) / len(vectors)
  else:
    vector_size = vectorizer.vector_size
    wv_res = np.zeros(vector_size)
    return wv_res

# 2. Carregamento dos Dados

## 2.1 Importação

In [6]:
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

## 2.2 Sample

In [7]:
df_sample = df.sample(frac=0.5, random_state=42).reset_index(drop=True) #validar numa base  menor

# 3. Descrição dos Dados

## 3.1 Tipos de Dados

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


In [9]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10536 entries, 0 to 10535
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         10536 non-null  int64 
 1   data_abertura         10536 non-null  object
 2   categoria             10536 non-null  object
 3   descricao_reclamacao  10536 non-null  object
dtypes: int64(1), object(3)
memory usage: 329.4+ KB


In [10]:
df_sample.dropna(inplace=True)
df.dropna(inplace=True)

In [11]:
df_sample.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,2505290,2017-06-08T12:00:00-05:00,Serviços de conta bancária,Oxxxx xxxx pagou o JP Morgan Chase {$ 1200.00}...
1,3625355,2020-04-27T12:00:00-05:00,Roubo / Relatório de disputa,Perdi meu cartão de crédito (uma reserva de sa...
2,1405636,2015-06-04T12:00:00-05:00,Hipotecas / Empréstimos,"Em 2005, fui para xxxx xxxx xxxx. Eles me dire..."
3,2915672,2018-05-23T12:00:00-05:00,Roubo / Relatório de disputa,Eu me inscrevi e recebi um cartão de crédito p...
4,1733697,2016-01-08T12:00:00-05:00,Roubo / Relatório de disputa,"Com o Chase A {$ 620,00} Charge (xxxx 2015) fo..."


In [12]:
# Obter os valores únicos e a quantidade de ocorrências coluna Categoria
valores_unicos = df_sample['categoria'].value_counts()
print(valores_unicos)

categoria
Serviços de conta bancária             2541
Cartão de crédito / Cartão pré-pago    2520
Roubo / Relatório de disputa           2420
Hipotecas / Empréstimos                1932
Outros                                 1123
Name: count, dtype: int64


# 4. Preparação dos Dados (Pré-Processamento)

In [13]:
# Stopwords

# Stopwords NLTK
stops_nltk = nltk.corpus.stopwords.words('portuguese')

# stopwords SpaCy
stops_spacy = nlp_stops.Defaults.stop_words

# Stopwords do SpaCy e NLTK combinadas
stops = list(set(stops_spacy).union(set(stops_nltk)))
len(stops)

500

In [14]:
# Texto
df_sample['texto'] = df_sample['descricao_reclamacao']

# Normalização
df_sample['texto_norm'] = df_sample.texto.apply(fn_limpa_texto)

# Tokenização
df_sample['tokens'] = df_sample.texto_norm.apply(word_tokenize)

# Lematização Completa
#df_sample['texto_lemma'] = df_sample.texto_norm.apply(fn_lematiza_texto) #desconsiderada lematização completa pela demora em tempo de processamento

# Lematização com Verbos
df_sample['texto_lemma_verb'] = df_sample.texto_norm.apply(fn_lematiza_verb_texto)

# Stemização
df_sample['texto_stemm'] = df_sample.tokens.apply(fn_stem)

# Drop Colunas desnecessárias
df_sample = df_sample.drop(columns=['id_reclamacao','descricao_reclamacao'])

In [15]:
# Comparação entre os tratamentos
print(df_sample.iloc[1]['texto'])
print('----------')
print(df_sample.iloc[1]['texto_norm'])
print('----------')
#print(df_sample.iloc[1]['texto_lemma'])
print('----------')
print(df_sample.iloc[1]['texto_lemma_verb'])
print('----------')
print(df_sample.iloc[1]['texto_stemm'])

Perdi meu cartão de crédito (uma reserva de safira de perseguição) entre a noite de XX/XX/2020 até o início da manhã de XX/XX/2020. Fiz uma compra durante esse período em xxxx xxxx xxxx com a quantidade de {$ 19,00}. Liguei para o Chase assim que descobri que meu cartão foi perdido, que fica em xx/xx/2020, e também percebeu que uma taxa de fraude de {$ 43,00} em xxxx xxxx xxxx xxxx foi postada na minha conta em xx/xx/2020 . Então, eu informei a Chase que não foi autorizado e eles me ajudaram a tirar isso da minha declaração.
No entanto, recebi uma carta no início do XXXX que sou responsável pela transação. De acordo com a letra única que Chase me enviou: "Recebi o benefício dessa transação". Recebimento ou coisas como vigilância de segurança que podem provar quem estava usando o cartão. Eu já contestei outras transações antes, mas outros agentes o levaram a sério e me enviaram coisas como recibos assinados para fazer backup da reivindicação.
Liguei para Chase novamente em XX/XX/2020, u

In [16]:
df_sample.head()

,data_abertura,categoria,texto,texto_norm,tokens,texto_lemma_verb,texto_stemm
0,2017-06-08T12:00:00-05:00,Serviços de conta bancária,Oxxxx xxxx pagou o JP Morgan Chase {$ 1200.00}...,oxxxx xxxx pagou morgan chase para pagamento h...,"[oxxxx, xxxx, pagou, morgan, chase, para, paga...",oxxxx xxxx pagar morgan chase para pagamento h...,oxxxx xxxx pag morgan chas par pag hipotec xxx...
1,2020-04-27T12:00:00-05:00,Roubo / Relatório de disputa,Perdi meu cartão de crédito (uma reserva de sa...,perdi meu cartão crédito uma reserva safira pe...,"[perdi, meu, cartão, crédito, uma, reserva, sa...",perdi meu cartão créditar uma reserva safira p...,perd meu cart crédit uma reserv saf persegu en...
2,2015-06-04T12:00:00-05:00,Hipotecas / Empréstimos,"Em 2005, fui para xxxx xxxx xxxx. Eles me dire...",fui para xxxx xxxx xxxx eles direcionaram banc...,"[fui, para, xxxx, xxxx, xxxx, eles, direcionar...",fui para xxxx xxxx xxxx eles direcionar banco ...,fui par xxxx xxxx xxxx ele direcion banc morga...
3,2018-05-23T12:00:00-05:00,Roubo / Relatório de disputa,Eu me inscrevi e recebi um cartão de crédito p...,inscrevi recebi cartão crédito preferido por s...,"[inscrevi, recebi, cartão, crédito, preferido,...",inscrevi recebi cartão crédito preferer por sa...,inscrev receb cart crédit prefer por sapph cha...
4,2016-01-08T12:00:00-05:00,Roubo / Relatório de disputa,"Com o Chase A {$ 620,00} Charge (xxxx 2015) fo...",com chase charge xxxx foi originalmente contes...,"[com, chase, charge, xxxx, foi, originalmente,...",com chase charge xxxx foi originalmente contes...,com chas charg xxxx foi orig contest dev nom c...


In [17]:
# Dividindo os dados em conjuntos de treinamento e teste
df_train, df_test = train_test_split(df_sample, test_size=0.3, random_state=42)

In [18]:
df_train.head()

,data_abertura,categoria,texto,texto_norm,tokens,texto_lemma_verb,texto_stemm
87,2018-08-29T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Eu tenho tentado descobrir mais sobre a conta ...,tenho tentado descobrir mais sobre conta cobra...,"[tenho, tentado, descobrir, mais, sobre, conta...",tenho tentar descobrir mais sobre conta cobran...,tenh tent descobr mais sobr cont cobranç cart ...
3550,2020-01-18T12:00:00-05:00,Serviços de conta bancária,Xx/xx/xxxxthe mais longe que cheguei com a fil...,xxxxthe mais longe que cheguei com filial xxxx...,"[xxxxthe, mais, longe, que, cheguei, com, fili...",xxxxthe mais longe que cheguei com filial xxxx...,xxxxth mais long que cheg com fil xxxx chas ba...
4002,2020-11-19T12:00:00-05:00,Outros,Eu perguntei em um filhote à venda em um site ...,perguntei filhote venda site line animais esti...,"[perguntei, filhote, venda, site, line, animai...",perguntar filhote venda site line animais esti...,pergunt filhot vend sit lin animal estim xxxx ...
9158,2017-10-30T12:00:00-05:00,Outros,Eu tenho tentado fazer o pagamento do meu carr...,tenho tentado fazer pagamento meu carro com ch...,"[tenho, tentado, fazer, pagamento, meu, carro,...",tenho tentar fazer pagamento meu carro com cha...,tenh tent faz pag meu carr com chas bank mais ...
403,2017-02-07T12:00:00-05:00,Serviços de conta bancária,Fui a perseguir e disse a eles a dívida xxxx d...,fui perseguir disse eles dívida xxxx financiam...,"[fui, perseguir, disse, eles, dívida, xxxx, fi...",fui perseguir dizer eles dívida xxxx financiam...,fui persegu diss ele dív xxxx financ xxxx não ...


In [19]:
df_test.head()

,data_abertura,categoria,texto,texto_norm,tokens,texto_lemma_verb,texto_stemm
518,2017-11-03T12:00:00-05:00,Serviços de conta bancária,"Saudações, tivemos várias contas bancárias do ...",saudações tivemos várias contas bancárias chas...,"[saudações, tivemos, várias, contas, bancárias...",saudações ter várias contas bancárias chase po...,saud tiv vár cont bancár chas por vár déc est ...
439,2017-02-14T12:00:00-05:00,Outros,A Chase está me cobrando taxas atrasadas há tr...,chase está cobrando taxas atrasadas três anos ...,"[chase, está, cobrando, taxas, atrasadas, três...",chase está cobrar taxas atrasar três anos acim...,chas est cobr tax atras trê ano acim por pag r...
5235,2016-01-22T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Atualmente, tenho uma disputa de cobrança em r...",atualmente tenho uma disputa cobrança relação ...,"[atualmente, tenho, uma, disputa, cobrança, re...",atualmente ter uma disputa cobrança relação me...,atual tenh uma disput cobranç relaç meu cart c...
6625,2020-12-15T12:00:00-05:00,Roubo / Relatório de disputa,Usei meu cartão de crédito para fazer uma comp...,usei meu cartão crédito para fazer uma compra ...,"[usei, meu, cartão, crédito, para, fazer, uma,...",usar meu cartão crédito para fazer uma compra ...,use meu cart crédit par faz uma compr xxxx par...
2698,2018-01-20T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Entrei em contato com todos os três relatórios...,entrei contato com todos três relatórios crédi...,"[entrei, contato, com, todos, três, relatórios...",entrar contato com todos três relatórios crédi...,entr contat com tod trê relatóri crédit inform...


# Aplicação das Técnicas de NLP

Faça aqui as demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

## Modelo 1 - Contagem de Termos Simples (Bag of Words)

In [20]:
# Vetorização
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) #sem stopwords

# Definição das Variáveis
text_vect_train = vect.fit_transform(df_train.texto) # Texto sem Pré-Processamento
x_train = text_vect_train
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train, y_train)

# Testar Modelo
x_test = vect.transform(df_test.texto)
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8997152799746916
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.88      0.94      0.91       714
            Hipotecas / Empréstimos       0.94      0.88      0.91       615
                             Outros       0.88      0.85      0.86       322
       Roubo / Relatório de disputa       0.89      0.88      0.89       735
         Serviços de conta bancária       0.90      0.93      0.91       775

                           accuracy                           0.90      3161
                          macro avg       0.90      0.89      0.90      3161
                       weighted avg       0.90      0.90      0.90      3161



In [21]:
# Vetorização
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) #sem stopwords

# Definição das Variáveis
text_vect_train = vect.fit_transform(df_train.texto_norm) # Texto Normalizado
x_train = text_vect_train
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train, y_train)

# Testar Modelo
x_test = vect.transform(df_test.texto_norm)
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.902562480227776
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.89      0.94      0.91       714
            Hipotecas / Empréstimos       0.94      0.88      0.91       615
                             Outros       0.89      0.85      0.87       322
       Roubo / Relatório de disputa       0.91      0.88      0.89       735
         Serviços de conta bancária       0.89      0.93      0.91       775

                           accuracy                           0.90      3161
                          macro avg       0.90      0.90      0.90      3161
                       weighted avg       0.90      0.90      0.90      3161



In [22]:
# Vetorização
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) #sem stopwords

# Definição das Variáveis
text_vect_train = vect.fit_transform(df_train.texto_lemma_verb) # Texto com Verbos Lematizados
x_train = text_vect_train
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train, y_train)

# Testar Modelo
x_test = vect.transform(df_test.texto_lemma_verb)
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8968680797216071
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.88      0.93      0.91       714
            Hipotecas / Empréstimos       0.94      0.88      0.91       615
                             Outros       0.86      0.85      0.86       322
       Roubo / Relatório de disputa       0.90      0.87      0.89       735
         Serviços de conta bancária       0.89      0.92      0.90       775

                           accuracy                           0.90      3161
                          macro avg       0.90      0.89      0.89      3161
                       weighted avg       0.90      0.90      0.90      3161



In [23]:
# Vetorização
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) #sem stopwords

# Definição das Variáveis
text_vect_train = vect.fit_transform(df_train.texto_stemm) # Texto com Stemização
x_train = text_vect_train
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train, y_train)

# Testar Modelo
x_test = vect.transform(df_test.texto_stemm)
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8826320784561847
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.89      0.90      0.90       714
            Hipotecas / Empréstimos       0.93      0.88      0.90       615
                             Outros       0.79      0.80      0.80       322
       Roubo / Relatório de disputa       0.88      0.87      0.87       735
         Serviços de conta bancária       0.89      0.91      0.90       775

                           accuracy                           0.88      3161
                          macro avg       0.87      0.87      0.87      3161
                       weighted avg       0.88      0.88      0.88      3161



Visto que o melhor resultado obtido foi com Verbos lematizados seguiremos com ele para os próximos modelos.

## Modelo 2 - Term Frequency (TF)

In [24]:
# Vetorização
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=False, norm='l1', stop_words=stops)

# Definição das Variáveis
text_vect_train = vect.fit_transform(df_train.texto_lemma_verb)
x_train = text_vect_train
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train, y_train)

# Testar Modelo
x_test = vect.transform(df_test.texto_lemma_verb)
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8070230939576084
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.81      0.85      0.83       714
            Hipotecas / Empréstimos       0.90      0.79      0.84       615
                             Outros       0.96      0.42      0.59       322
       Roubo / Relatório de disputa       0.74      0.82      0.78       735
         Serviços de conta bancária       0.79      0.93      0.85       775

                           accuracy                           0.81      3161
                          macro avg       0.84      0.76      0.78      3161
                       weighted avg       0.82      0.81      0.80      3161



## Modelo 3 - Inverse Document Frequency (TF-IDF)

In [25]:
# Vetorização
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)

# Definição das Variáveis
text_vect_train = vect.fit_transform(df_train.texto_lemma_verb)
x_train = text_vect_train
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train, y_train)

# Testar Modelo
x_test = vect.transform(df_test.texto_lemma_verb)
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9035115469788042
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.88      0.93      0.91       714
            Hipotecas / Empréstimos       0.93      0.89      0.91       615
                             Outros       0.91      0.81      0.86       322
       Roubo / Relatório de disputa       0.90      0.90      0.90       735
         Serviços de conta bancária       0.90      0.93      0.91       775

                           accuracy                           0.90      3161
                          macro avg       0.91      0.89      0.90      3161
                       weighted avg       0.90      0.90      0.90      3161



## Modelo 4 - Embeddings com Word2Vec

Para esta aplicação estaremos utilizando o repositório Word Embeddings Pré-treinadas em Português (Word2VEc) com 300 dimensões, tanto Cbow como Skipgram

Repositório original: http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

In [26]:
# Modelo SkipGram
model_skip = KeyedVectors.load_word2vec_format('skip_s300.txt')
vectorizer = model_skip

# Definição das Variáveis
df_train["vetor"] = df_train["tokens"].apply(average_vector) #transformando texto para embeddings
x_train = df_train["vetor"]
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(list(x_train), y_train)

# Testar Modelo
df_test["vetor"] = df_test["tokens"].apply(average_vector) #transformando texto para embeddings
x_test = df_test["vetor"]
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(list(x_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8190446061372983
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.85      0.85      0.85       714
            Hipotecas / Empréstimos       0.84      0.82      0.83       615
                             Outros       0.81      0.62      0.70       322
       Roubo / Relatório de disputa       0.77      0.81      0.79       735
         Serviços de conta bancária       0.83      0.89      0.86       775

                           accuracy                           0.82      3161
                          macro avg       0.82      0.80      0.80      3161
                       weighted avg       0.82      0.82      0.82      3161



In [27]:
# Modelo Cbow
model_cbow = KeyedVectors.load_word2vec_format('cbow_s300.txt')
vectorizer = model_cbow

# Definição das Variáveis
df_train["vetor"] = df_train["tokens"].apply(average_vector) #transformando texto para embeddings
x_train = df_train["vetor"]
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(list(x_train), y_train)

# Testar Modelo
df_test["vetor"] = df_test["tokens"].apply(average_vector) #transformando texto para embeddings
x_test = df_test["vetor"]
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(list(x_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8177791838025941
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.86      0.85      0.85       714
            Hipotecas / Empréstimos       0.84      0.80      0.82       615
                             Outros       0.80      0.62      0.70       322
       Roubo / Relatório de disputa       0.76      0.81      0.79       735
         Serviços de conta bancária       0.82      0.89      0.85       775

                           accuracy                           0.82      3161
                          macro avg       0.82      0.79      0.80      3161
                       weighted avg       0.82      0.82      0.82      3161



## Modelo 5 - Embeddings Próprio (do contexto do Dataset)

https://radimrehurek.com/gensim/models/word2vec.html

In [28]:
# Criando Espaço Vetorial com 100 dimensões com o nosso vocabulário
w2v_df_sample = Word2Vec(df_train['tokens'], vector_size=100, min_count=5, sg=0)

#sg – Training algorithm: 1 for skip-gram; otherwise CBOW.
#min_count (int, optional) – Ignores all words with total frequency lower than this.

#Note que deve ser utilizado df de treino, teste será aplicado para validar este espaço vetorial.

In [29]:
# Modelo Cbow com Contexto Próprio
model_cbow = w2v_df_sample.wv #'.wv' é um atributo do modelo Word2Vec que contém os vetores das palavras.
vectorizer = model_cbow

# Definição das Variáveis
df_train["vetor"] = df_train["tokens"].apply(average_vector) #transformando texto para embeddings
x_train = df_train["vetor"]
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(list(x_train), y_train)

# Testar Modelo
df_test["vetor"] = df_test["tokens"].apply(average_vector) #transformando texto para embeddings
x_test = df_test["vetor"]
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(list(x_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8582726985131287
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.86      0.86      0.86       714
            Hipotecas / Empréstimos       0.90      0.86      0.88       615
                             Outros       0.85      0.79      0.82       322
       Roubo / Relatório de disputa       0.81      0.83      0.82       735
         Serviços de conta bancária       0.88      0.90      0.89       775

                           accuracy                           0.86      3161
                          macro avg       0.86      0.85      0.85      3161
                       weighted avg       0.86      0.86      0.86      3161



## Modelo 6 - Sentence Transformers

O Sentence-Transformers é uma biblioteca construída sobre o framework Transformers, e foi projetada para a geração de representações semânticas de sentenças (documentos ou pedaços de texto) de alta qualidade.

Vamos utilizar o modelo 'distiluse-base-multilingual-cased-v2', que é um modelo da família DistilBERT (uma versão mais leve e eficiente do BERT) que foi pré-treinado especificamente para a geração de embeddings de sentenças multilíngues.

In [37]:
# Vetorização
st = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2') #Modelo com Espaço vetorial denso de 512 dimensões

df_sample['vetor_embedding'] = list(st.encode(df_sample['texto_norm']))

df_train_st, df_test_st = train_test_split(df_sample, test_size=0.3, random_state=42)

# Definição das Variáveis
x_train_st = df_train_st['vetor_embedding'].to_list()
y_train_st = df_train_st['categoria'].to_list()

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(list(x_train_st), y_train_st)

# Testar Modelo
x_test_st = df_test_st['vetor_embedding'].to_list()
y_test_st = df_test_st['categoria'].to_list()

# Avaliação do Modelo
y_pred = model.predict(list(x_test_st))
print(accuracy_score(y_test_st, y_pred))
print(classification_report(y_test_st, y_pred))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.7617842454919329
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.75      0.77      0.76       714
            Hipotecas / Empréstimos       0.84      0.80      0.82       615
                             Outros       0.71      0.66      0.68       322
       Roubo / Relatório de disputa       0.70      0.70      0.70       735
         Serviços de conta bancária       0.79      0.83      0.81       775

                           accuracy                           0.76      3161
                          macro avg       0.76      0.75      0.75      3161
                       weighted avg       0.76      0.76      0.76      3161



## Modelo 7 - Bert Sequence Classifier

In [38]:
# Tokenizando os textos

# Inicializando o tokenizador BERT para o português
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Seperação dos dados de treino e teste
## Dados de treino
train_texts = df_train['texto_norm'].tolist()
train_labels = df_train['categoria'].tolist()
## Dados de teste
test_texts = df_test['texto_norm'].tolist()
test_labels = df_test['categoria'].tolist()

# Codificando os textos usando o tokenizador
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt', max_length=128)

# Codificando as categorias em números e convertendo as categorias para IDs numéricos
le = preprocessing.LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

# Convertendo os dados para tensores PyTorch
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [39]:
# Inicializando o modelo BERT pré-treinado para classificação
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=len(set(train_labels)))

# Inicializando o otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Inicializando o DataLoader para treinamento e teste
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)#test pode remover o batch size, apenas aplicavel para train

# Treinamento do modelo
## identifica qual o processador da sessão (CPU ou GPU?), recomendo usar GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 3

for epoch in range(num_epochs):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 3: 100%|██████████| 1844/1844 [03:20<00:00,  9.18it/s]


In [40]:
# Avaliação do modelo
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Avaliação do Modelo
y_pred = le.inverse_transform(all_predictions)
y_test = le.inverse_transform(all_labels)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Evaluating: 100%|██████████| 791/791 [00:23<00:00, 34.39it/s]


0.817462828218918
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.89      0.70      0.78       714
            Hipotecas / Empréstimos       0.89      0.84      0.86       615
                             Outros       0.72      0.84      0.77       322
       Roubo / Relatório de disputa       0.79      0.82      0.81       735
         Serviços de conta bancária       0.79      0.90      0.84       775

                           accuracy                           0.82      3161
                          macro avg       0.82      0.82      0.81      3161
                       weighted avg       0.82      0.82      0.82      3161



## Modelo Escolhido

Baseado nos testes que foram realizados, devido a boa performance F1, acurácia e também tempo de processamento foi selecionado o modelo:

- Modelo 3 - TF-IDF (texto_lemma_verb)

# Solução Proposta

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

O modelo precisar atingir um score na métrica F1 Score superior a 75%.

Seguimos o seguinte processo:

Data Loader > Text Normalization > Text Vectorization > Estimator

Código desenvolvido para rodar em Google Colaboraty.

In [41]:
# Importando Ambiente
!python -m spacy download pt_core_news_lg --quiet
!python -m spacy download pt_core_news_sm --quiet
!pip install nltk --quiet
!pip install gensim==4.3.2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 74.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [42]:
# Importando Bibliotecas

# Pacotes Básicos
import pandas as pd
import numpy as np
import string
from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
from sklearn.model_selection import train_test_split

# Pacotes NLP
import nltk
import pickle
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from nltk.stem.rslp import RSLPStemmer
rslp = RSLPStemmer()
import spacy
import pt_core_news_lg
import pt_core_news_sm
nlp_stops = spacy.load('pt_core_news_sm')
nlp = spacy.load('pt_core_news_lg')

# Algoritmos de Vetorização NLP
from sklearn.feature_extraction.text import TfidfVectorizer #TF/TF-IDF
from sklearn.feature_extraction.text import CountVectorizer #BoW
from gensim.models import KeyedVectors #Word2Vec

# Algoritmos de Classificação
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

# Avaliação do Modelo
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [43]:
# Importando Funções Auxiliares

# Função Lematização Completa do Documento
def fn_lematiza_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        sent.append(word.lemma_)
    return ' '.join(sent)

# Função Lematização dos Verbos do Documento
def fn_lematiza_verb_texto(texto):
    sent = []
    doc = nlp(texto)
    for word in doc:
        if word.pos_ =='VERB':
            sent.append(word.lemma_)
        else:
            sent.append(word.text)
    return " ".join(sent)

# Função Stemização
def fn_stem(texto):
  return ' '.join([rslp.stem(token) for token in texto])

# Função para Limpar documento
def fn_limpa_texto(texto):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    texto = texto.translate(table)
    texto = texto.strip().lower()
    texto = "".join([w for w in texto if not w.isdigit()])
    texto = word_tokenize(texto)
    texto = [y for y in texto if len(y) > 2]
    return " ".join([t for t in texto])
    return texto

In [44]:
# Carregando os Dados
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')
df.dropna(inplace=True)

In [45]:
# Normalização dos Dados
# Stopwords NLTK
stops_nltk = nltk.corpus.stopwords.words('portuguese')

# stopwords SpaCy
stops_spacy = nlp_stops.Defaults.stop_words

# Stopwords do SpaCy e NLTK combinadas
stops = list(set(stops_spacy).union(set(stops_nltk)))

# Texto
df['texto'] = df['descricao_reclamacao']

# Normalização
df['texto_norm'] = df.texto.apply(fn_limpa_texto)

# Tokenização
df['tokens'] = df.texto_norm.apply(word_tokenize)

# Lematização com Verbos
df['texto_lemma_verb'] = df.texto_norm.apply(fn_lematiza_verb_texto)

# Drop Colunas desnecessárias
df = df.drop(columns=['id_reclamacao','descricao_reclamacao'])

In [46]:
# Aplicando Modelo Selecionado

# Dividindo os dados em conjuntos de treinamento e teste
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)

# Vetorização
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)

# Definição das Variáveis
text_vect_train = vect.fit_transform(df_train.texto_lemma_verb)
x_train = text_vect_train
y_train = df_train["categoria"]

# Treinar Modelo
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train, y_train)

# Testar Modelo
x_test = vect.transform(df_test.texto_lemma_verb)
y_test = df_test["categoria"]

# Avaliação do Modelo
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.909832953682612
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.92      0.91      0.92      1290
            Hipotecas / Empréstimos       0.91      0.93      0.92       922
                             Outros       0.91      0.85      0.88       549
       Roubo / Relatório de disputa       0.89      0.89      0.89      1204
         Serviços de conta bancária       0.92      0.93      0.93      1303

                           accuracy                           0.91      5268
                          macro avg       0.91      0.90      0.91      5268
                       weighted avg       0.91      0.91      0.91      5268



In [47]:
# Salvando o Modelo e a Vetorização
!ls -la
pickle.dump(model, open('modelo_classificacao.pkl', 'wb'))
!ls -la
pickle.dump(vect, open('vetorizador.pkl', 'wb'))

total 7030920
drwxr-xr-x 1 root root       4096 Jun 23 14:47 .
drwxr-xr-x 1 root root       4096 Jun 23 13:15 ..
-rw-r--r-- 1 root root 2656045531 Oct  4  2018 cbow_s300.txt
-rw-r--r-- 1 root root  929305948 Feb 16  2021 cbow_s300.zip
drwxr-xr-x 4 root root       4096 Jun 20 18:45 .config
drwxr-xr-x 2 root root       4096 Jun 23 14:39 .ipynb_checkpoints
drwxr-xr-x 1 root root       4096 Jun 20 18:46 sample_data
-rw-r--r-- 1 root root 2655642222 Oct  4  2018 skip_s300.txt
-rw-r--r-- 1 root root  958619745 Nov 30  2023 skip_s300.zip
total 7031996
drwxr-xr-x 1 root root       4096 Jun 23 16:00 .
drwxr-xr-x 1 root root       4096 Jun 23 13:15 ..
-rw-r--r-- 1 root root 2656045531 Oct  4  2018 cbow_s300.txt
-rw-r--r-- 1 root root  929305948 Feb 16  2021 cbow_s300.zip
drwxr-xr-x 4 root root       4096 Jun 20 18:45 .config
drwxr-xr-x 2 root root       4096 Jun 23 14:39 .ipynb_checkpoints
-rw-r--r-- 1 root root    1099324 Jun 23 16:00 modelo_classificacao.pkl
drwxr-xr-x 1 root root       4096 J

In [48]:
# Abrindo o Modelo e Vetorizador
!ls -la
classificador_modelo_selecionado = pickle.load(open('modelo_classificacao.pkl', 'rb'))
vetorizador_modelo_selecionado = pickle.load(open('vetorizador.pkl', 'rb'))

total 7032816
drwxr-xr-x 1 root root       4096 Jun 23 16:00 .
drwxr-xr-x 1 root root       4096 Jun 23 13:15 ..
-rw-r--r-- 1 root root 2656045531 Oct  4  2018 cbow_s300.txt
-rw-r--r-- 1 root root  929305948 Feb 16  2021 cbow_s300.zip
drwxr-xr-x 4 root root       4096 Jun 20 18:45 .config
drwxr-xr-x 2 root root       4096 Jun 23 14:39 .ipynb_checkpoints
-rw-r--r-- 1 root root    1099324 Jun 23 16:00 modelo_classificacao.pkl
drwxr-xr-x 1 root root       4096 Jun 20 18:46 sample_data
-rw-r--r-- 1 root root 2655642222 Oct  4  2018 skip_s300.txt
-rw-r--r-- 1 root root  958619745 Nov 30  2023 skip_s300.zip
-rw-r--r-- 1 root root     838798 Jun 23 16:00 vetorizador.pkl


In [49]:
# Executando o Modelo
texto_teste = """ Perdi meu cartão de crédito (uma reserva de safira de perseguição) entre a noite de XX/XX/2020 até o início da manhã de XX/XX/2020. Fiz uma compra
durante esse período em xxxx xxxx xxxx com a quantidade de {$ 19,00}. Liguei para o Chase assim que descobri que meu cartão foi perdido, que fica em xx/xx/2020, e
também percebeu que uma taxa de fraude de {$ 43,00} em xxxx xxxx xxxx xxxx foi postada na minha conta em xx/xx/2020 . Então, eu informei a Chase
que não foi autorizado e eles me ajudaram a tirar isso da minha declaração. """

texto_vetor = vetorizador_modelo_selecionado.transform([texto_teste])

texto_classificador = model.predict(texto_vetor)

classificador_modelo_selecionado.predict(texto_vetor)

print("A categoria do texto é:", texto_classificador[0])

A categoria do texto é: Roubo / Relatório de disputa


In [8]:
# Baixando os Requirements
!pip freeze > requirements.txt